# LSL data processing

In [1]:
# Importing Python and external packages


import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

import scipy
from scipy import stats
from scipy import signal
from scipy.signal import spectrogram, butter, filtfilt, freqz
from scipy.signal.windows import hann

import seaborn as sns
import pingouin as pg
from itertools import combinations
#from statannotations.Annotator import Annotator

import openpyxl
from openpyxl import Workbook, load_workbook
# import xlrd
import pickle
import json
import csv
import fooof
from fooof.plts.spectra import plot_spectrum

#mne
import mne
from mne.time_frequency import tfr_morlet 

# xdf
import pyxdf
import pyxdftools
from pyxdftools import XdfData

import datetime


/var/folders/8d/3m_ljz890_j62q7hb7mpzdc40000gn/T/ipykernel_49450/509795276.py:32: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  import fooof


In [4]:
%matplotlib qt

In [3]:


jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Patterned_stimulation_project', 'patterned_DBS')
sys.path.append(project_path)

os.chdir(project_path)

# patterned DBS
import src.patterned_DBS.beta_suppression_latency.patterned_dbs_pilot as pilot
import src.patterned_DBS.beta_suppression_latency.patterned_find_dbs_OFF_moment as find_dbs_OFF_moment

# utility functions
import src.patterned_DBS.utils.io as io
import src.patterned_DBS.utils.find_folders as find_folders
import src.patterned_DBS.utils.lfp_preprocessing as lfp_preprocessing
import src.patterned_DBS.utils.xdf_structure as xdf_structure

importlib.reload(pilot)
importlib.reload(find_dbs_OFF_moment)
importlib.reload(io)
importlib.reload(find_folders)
importlib.reload(lfp_preprocessing)
importlib.reload(xdf_structure)

<module 'src.patterned_DBS.utils.xdf_structure' from '/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/xdf_structure.py'>

## 1. Load xdf file

In [12]:
xdf_structured_data = xdf_structure.get_xdf_structure(
    sub="084",
    stimulation="StimOnB",
    medication="on", 
    task="updrs", 
    run="1",
)

Stream 2: Calculated effective sampling rate 564.1455 Hz is different from specified rate 10.0000 Hz.


In [13]:
xdf_structured_data["ultraleap_data"]

,left_pinky_tip_x,left_pinky_tip_y,left_pinky_tip_z,left_ring_tip_x,left_ring_tip_y,left_ring_tip_z,left_middle_tip_x,left_middle_tip_y,left_middle_tip_z,left_index_tip_x,...,right_index_tip_x,right_index_tip_y,right_index_tip_z,right_thumb_tip_x,right_thumb_tip_y,right_thumb_tip_z,right_palm_x,right_palm_y,right_palm_z,timestamps
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143694e+06
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143694e+06
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143694e+06
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143694e+06
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.143694e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.144387e+06
390604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.144387e+06
390605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.144387e+06
390606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.144387e+06


In [14]:
xdf_structured_data["SAGA_data"]

,BIP 01,BIP 02,BIP 03,X,Y,Z,X,Y,Z,ISO aux,ISO aux,AUX 3-3,TRIGGERS,STATUS,COUNTER,timestamps
0,9865.063477,0.0,-5403.250977,-0.802114,0.572265,0.398327,-0.669116,-0.734828,0.179361,0.036238,0.051629,0.0,255.0,1026.0,1269442.0,3.143704e+06
1,9863.502930,0.0,-5394.515625,-0.797795,0.573394,0.397929,-0.670373,-0.731752,0.179203,0.035732,0.051308,0.0,255.0,1026.0,1269443.0,3.143704e+06
2,9856.749023,0.0,-5383.669434,-0.795837,0.573278,0.395609,-0.666448,-0.735422,0.176210,0.035645,0.051243,0.0,255.0,1026.0,1269444.0,3.143704e+06
3,9851.189453,0.0,-5375.704590,-0.795402,0.573004,0.397809,-0.662603,-0.743848,0.175910,0.035651,0.051535,0.0,255.0,1026.0,1269445.0,3.143704e+06
4,9850.161133,0.0,-5374.456543,-0.791188,0.572622,0.399223,-0.661841,-0.742753,0.170045,0.035606,0.051423,0.0,255.0,1026.0,1269446.0,3.143704e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685095,23031.062500,0.0,-6997.672852,0.092755,0.790863,-0.461171,-0.537850,-0.836803,-0.091851,0.036092,0.051065,0.0,255.0,1026.0,3954537.0,3.144375e+06
2685096,22407.003906,0.0,-6933.661133,0.094156,0.784033,-0.466934,-0.538785,-0.834619,-0.092966,0.036101,0.051183,0.0,255.0,1026.0,3954538.0,3.144375e+06
2685097,21973.787109,0.0,-6884.257812,0.092389,0.783591,-0.466742,-0.544113,-0.828339,-0.097359,0.036231,0.051251,0.0,255.0,1026.0,3954539.0,3.144375e+06
2685098,21651.011719,0.0,-6844.470215,0.094351,0.784334,-0.472550,-0.537493,-0.832374,-0.093951,0.036060,0.051371,0.0,255.0,1026.0,3954540.0,3.144375e+06
